# 3 - Modelagem

Concluída a fase de exploração de dados, vamos agora modelá-los usando os padrões descobertos.

Nessa fase, buscaremos construir um modelo que, baseado nas variáveis contidas na base de dados de COVID, prediga o resultado de um teste para COVID.

In [14]:
# para carregar a base de dadas limpa
import pickle
from typing import Tuple

import numpy as np
import pandas as pd
from scipy import stats as spst
from tqdm import tqdm

# gráficos
import seaborn as sns
from matplotlib import rcParams, pyplot as plt

# pacote com funções para análise desse projeto
import os
cwd = os.getcwd()
os.chdir("../")
import scripts.plots as splt, scripts.metrics as smetrics
os.chdir(cwd)

# parâmetros do matplotlib
# essencialmente, para deixar os gráficos maiores por padrão
rcParams['figure.dpi'] = 120
rcParams['figure.figsize'] = (10, 8)

## Importação e checagem de integridade

Vamos importar os dados limpos e verificar a integridade dos mesmos.

In [50]:
with open(r'../data/covid_limpo.df', 'rb') as covidfile:
    pickler = pickle.Unpickler(file = covidfile)
    covid = pickler.load()

In [22]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496285 entries, 0 to 499691
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   sex                  496285 non-null  category
 1   patient_type         496285 non-null  category
 2   pneumonia            496285 non-null  category
 3   age                  496285 non-null  int8    
 4   diabetes             496285 non-null  category
 5   copd                 496285 non-null  category
 6   asthma               496285 non-null  category
 7   inmsupr              496285 non-null  category
 8   hypertension         496285 non-null  category
 9   other_disease        496285 non-null  category
 10  cardiovascular       496285 non-null  category
 11  obesity              496285 non-null  category
 12  renal_chronic        496285 non-null  category
 13  tobacco              496285 non-null  category
 14  contact_other_covid  343718 non-null  category
 15  

In [23]:
covid.head()

,sex,patient_type,pneumonia,age,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res
id,,,,,,,,,,,,,,,,
0,0,1,0,27,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,24,0,0,0,0,0,0,0,0,0,0,NaN,1
2,1,0,0,54,0,0,0,0,0,0,0,1,0,0,NaN,1
3,0,0,1,30,0,0,0,0,0,0,0,0,0,0,NaN,1
4,1,0,0,60,1,0,0,0,1,0,1,0,0,0,NaN,1


In [24]:
covid.covid_res.value_counts(normalize = True)

0    0.558919
1    0.441081
Name: covid_res, dtype: float64

## Construção dos possíveis modelos

Vamos construir os possíveis modelos, para então compará-los.

Cada modelo será um `Pipeline` onde entra a base de treino (`X_train`, `y_train`) e sai uma probabilidade de classificação. Para cada modelo, construiremos os passos dessa `Pipeline`.

### 1. Tratamento dos dados faltantes

Como falamos na seção de análise exploratória dos dados, tínhamos três opções para tratamento dos dados faltantes no campo `contact_other_covid`:

1. Remover os registros correspondentes;
2. Codificar os dados faltantes como uma nova categoria; ou
3. Remover o campo da análise.

Vamos agora implementar os três processadores.

In [51]:
from sklearn.model_selection import train_test_split
# função para separar bases diferentes em treino e teste

def train_test_split_base(
    base: pd.DataFrame,
    test_size: float,
    target: str,
    stratify: bool = True
) -> Tuple:
    
    base = base.copy()

    X = base.drop(columns = [target])
    y = base[target]

    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size = test_size,
            stratify = y,     # não é estritamente necessário pois a base é razoavelmente balanceada
        )
    
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size = test_size,
        )

    return base, (X_train, X_test, y_train, y_test)

In [52]:
bases = {}
tts = {}   # train test split
test_size = 0.4


#### 1. Remover os registros correspondentes

In [53]:
alternativa = 'contact_dropna'

bases[alternativa], tts[alternativa] = train_test_split_base(
    base = covid.dropna(),
    test_size = test_size,
    target = 'covid_res',
    stratify = True,
)

#### 2. Codificar os dados faltantes como uma nova categoria

In [54]:
alternativa = 'contact_nacat'

base = covid.copy()
base['contact_other_covid'] = base['contact_other_covid'].cat.codes.astype('category')

bases[alternativa], tts[alternativa] = train_test_split_base(
    base = base,
    test_size = test_size,
    target = 'covid_res',
    stratify = True,
)

#### 3. Eliminar o campo `contact_other_covid` da análise

In [59]:
alternativa = 'contact_drop'

bases[alternativa], tts[alternativa] = train_test_split_base(
    base = covid.drop(columns = 'contact_other_covid'),
    test_size = test_size,
    target = 'covid_res',
    stratify = True,
)

### Modelo 1: blah blah